<a href="https://colab.research.google.com/github/CT-Cultures/Content/blob/master/RegOverview_Issue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [1] Setting up Environment and Install Requirements

## Set Working Directory, Install Requirements

In [ ]:
import os
import sys
# Set Path
path_wd = '/content/drive/MyDrive/Github/Content'
os.chdir(path_wd)

!pip install -r sources/ChinaFilm/requirements.txt
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

## Load and Initialize the Movie Registration Class

In [19]:
!#Load Libraries
import gc
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
from sources.ChinaFilm.Record_Registration import Registration # assume relative path at path_wd

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Initialize Registration Class
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
dy_reg = Registration(driver)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


## Load Latest Registration Publication

In [95]:
os.chdir('/content/drive/MyDrive/Github/Content/sources/ChinaFilm')
df = dy_reg.Refined_Records(update_first =False)
curr_issue_name = df.sort_values(['公示日期', '公示批次名称'], ascending=False)['公示批次名称'].iloc[0]
df_curr =  df.loc[df.公示批次名称 == curr_issue_name,]
df_curr.columns

Index(['电影类别', '备案立项号', '片名', '备案单位', '编剧', '备案结果', '备案地', '梗概', '公示日期',
       '公示批次名称', '备案详细页链接', '公示批次链接', '公示年', '公示批次起始', '类型', '备案申请年份',
       '备案立项年度顺序号', '公示覆盖期间', '公示覆盖天数'],
      dtype='object')

# [2] Generating Article Contents

In [66]:
df_type = df_curr.groupby('电影类别')['电影类别'].count().sort_values(ascending=False).rename('数量').to_frame().reset_index()

##2021年4月28日,3月整月的备案公示新鲜出炉！这一批次中，最遥远的项目是《幕后英雄》,备案号为影剧备字〔2021〕第1360号,最近期的项目是《熊猫传奇——黑洞之吻》,备案号为影特备字〔2021〕第006号。本次完成备案的共计故事影片229部、科教影片8部、纪录影片6部、动画影片6部以及特种影片1部。


In [70]:
# Obtain Variables
df_curr.loc[:,'公示日期'] = df_curr.loc[:,'公示日期'].astype('datetime64')
pub_year = df_curr.loc[0,'公示日期'].year
pub_month = df_curr.loc[0,'公示日期'].month
pub_day = df_curr.loc[0,'公示日期'].day
df_curr_sorted = df_curr.sort_values(['备案申请年份', '备案立项年度顺序号'])
df_type = df_curr.groupby('电影类别')['电影类别'].count().sort_values(ascending=False).rename('数量').to_frame().reset_index()

# Write Content
T1 = '{year}年{month}月{day}日,'.format(year=pub_year, month=pub_month, day=pub_day)
T1 += '{month}月{part_of_month}的备案公示新鲜出炉！'.format(month=df_curr.loc[0,'公示批次起始'][1], part_of_month=df_curr.loc[0,'公示批次起始'][2])
T1 += '这一批次中，最遥远的项目是《{}》,'.format(df_curr_sorted.loc[0, '片名']) 
T1 += '备案号为{},'.format(df_curr_sorted.loc[0, '备案立项号'])
T1 += '最近期的项目是《{}》,'.format(df_curr_sorted.loc[df_curr.shape[0]-1, '片名'])
T1 += '备案号为{}。'.format(df_curr_sorted.loc[df_curr.shape[0]-1, '备案立项号'])
T1 += '本次完成备案的共计'
for i, row in df_type.iterrows():
  if i == df_type.shape[0]-1:
    T1 = T1.rstrip('、')
    T1 += '以及{type}{n}部。'.format(type=row['电影类别'], n=row['数量'])
  else:
    T1 += '{type}{n}部、'.format(type=row['电影类别'], n=row['数量'])
print(T1)

2021年4月28日,3月整月的备案公示新鲜出炉！这一批次中，最遥远的项目是《幕后英雄》,备案号为影剧备字〔2021〕第1360号,最近期的项目是《熊猫传奇——黑洞之吻》,备案号为影特备字〔2021〕第006号。本次完成备案的共计故事影片229部、科教影片8部、纪录影片6部、动画影片6部以及特种影片1部。


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


##本批次中，获得上市影视公司参投的项目预计有10部,占比4.0%。AI福彩慧眼识别出的重点关注项目有：
《发明一个夏天》,编剧：高亦可
备案单位：中国电影股份有限公司
胡舟回到年少时度过夏天的小镇，回想起曾经的玩伴唐露，以及小镇上神秘的河流。他决定追溯过往，解开困扰自己多年的梦境。

In [151]:
# 本批次中与上市影视公司关联的项目有
info_public_film_co = pd.read_csv('/content/drive/MyDrive/Github/Content/tools/articles/reference/info_public_film_co.csv', index_col=0, encoding='utf-8-sig')
ls = info_public_film_co['公司简称'].apply(eval).sum()
pat_public = '|'.join(ls)
pat_public += '|阿里|腾讯|爱奇艺|英皇|寰亚|银都|美亚|大盛|儒意|灿星|横店'
df_focus = df_curr.loc[df_curr['备案单位'].str.contains(pat_public), ['电影类别', '片名', '备案单位', '编剧', '梗概', '备案地']]
#ids = [39,120,150, 170,230,231]
df_focus_narrowed = df_curr.loc[ids, ['电影类别', '片名', '备案单位', '编剧', '梗概', '备案地']]

# 
T2 = '本批次中，获得上市影视公司参投的项目预计有{}部,'.format(df_focus.shape[0])
T2 += '占比{}%。'.format(df_focus.shape[0]/df_curr.shape[0]*100)
T2 += 'AI福彩慧眼识别出的重点关注项目有:\n'
for _, row in df_focus_narrowed.iterrows():
  T2 += '\n《{name}》\n编剧：{writer}'.format(name=row['片名'],writer=row['编剧'])
  T2 += '\n备案单位：{co}'.format(co=row['备案单位'])
  T2 += '\n{synopsis}\n'.format(synopsis=row['梗概'])

print(T2)


本批次中，获得上市影视公司参投的项目预计有10部,占比4.0%。AI福彩慧眼识别出的重点关注项目有:

《发明一个夏天》
编剧：高亦可
备案单位：中国电影股份有限公司
胡舟回到年少时度过夏天的小镇，回想起曾经的玩伴唐露，以及小镇上神秘的河流。他决定追溯过往，解开困扰自己多年的梦境。

《计划外的姐弟恋》
编剧：吴冰、齐中旸
备案单位：青春光线(扬州)影业有限公司
32岁投行精英卓娅，秉承“计算一切”的理念生活，却在破除升职壁垒的路上，遇到实习生弟弟彦兮。面对彦兮恣意洒脱、浪漫痴情的追求，卓娅体会爱情，找到自我，突破成见，放弃原本的无爱婚姻计划，勇敢奔爱。

《日常警事》
编剧：李忱
备案单位：陕西广电影视文化产业发展有限公司、杭州东桥文化传媒有限公司、北京奥创世纪网络影视发行有限公司、北京爱奇艺科技有限公司
取材于警察英模李贝职业生涯中的一段时期。李贝自信果敢，一直期待投身一线，却阴差阳错成为新成立的巡处警大队的大队长。日复一日的基层工作使之与警员们建立起信任与友情，也让她重新正视自己的“英雄梦'。

《无价之宝》
编剧：韩家女
备案单位：上海阿里巴巴影业有限公司
因为一笔债务纠纷，六岁的许芊芊被押在石振邦手上作为“担保”，然而芊芊的父母再未将她接走，石振邦独自一人担起了养大许芊芊的责任。为了合法收养芊芊，石振邦从一个落魄中年努力成为了法律和道德上的合格父亲。

《二郎神》
编剧：邢嘉芸
备案单位：北京光线影业有限公司
为寻找讨伐猿魔时丢失的天目，杨戬来到凡间，意外遇见凡人何小天并收其为徒。为了找回天目两人在凡间经历了一系列让人啼笑皆非的故事。当杨戬再次面对猿魔的时候，他发现自己的劫数远没有丢失天目这么简单.

《土行孙之破土重生》
编剧：刘洋
备案单位：北京光线影业有限公司
生来不会法术的仙门弟子土行孙机缘之下学会了土行术，肩负起了看守封印的责任。漶本是被混元金斗镇压在地底，申公豹和张奎密谋盗走了混元金斗后，它便逃了出来。危机关头，土行孙决定牺牲自己，将漶再次封印了起来。



##本批次中，共有16家公司完成了两部或以上故事片或合拍片的备案，创造力满满。
中央宣传部电影卫星频道节目制作中心：《虫口夺粮》,《突破口》,《您别不爱听》,《地火》

In [174]:
df_2plus_co = df_curr[df_curr['电影类别'].str.contains(pat_film)].groupby(['备案单位'])['备案单位'].count().sort_values(ascending=False).rename('数量').to_frame().reset_index()
pat_2plus_co = '|'.join(df_2plus_co.loc[df_2plus_co['数量']>1, '备案单位'].to_list())
df_2plus = df_curr[df_curr['电影类别'].str.contains(pat_film) & df_curr['备案单位'].str.contains(pat_2plus_co)]
df_2plus = df_2plus.loc[:,['备案单位','片名', '编剧', '梗概']]
df_count = df_2plus.groupby('备案单位')['备案单位'].count().rename('备案数量').reset_index()
df_2plus = df_2plus.merge(df_count, on='备案单位').sort_values('备案数量', ascending=False)
df_2plus['片单'] = df_2plus.groupby('备案单位')['片名'].transform(lambda x: ','.join("《" + x +"》"))
df_2plus = df_2plus[['备案单位','片单']].drop_duplicates()

##
T3 = '本批次中，共有{n}家公司完成了两部或以上故事片或合拍片的备案，创造力满满。'.format(n=df_2plus.shape[0])
for _, row in df_2plus.iterrows():
  T3 += '\n{co}：'.format(co=row['备案单位'])
  T3 += '{}'.format(row['片单'])

print(T3)

本批次中，共有16家公司完成了两部或以上故事片或合拍片的备案，创造力满满。
中央宣传部电影卫星频道节目制作中心：《虫口夺粮》,《突破口》,《您别不爱听》,《地火》
苏州宸玮文化发展有限公司：《过期不候》,《蓝颜无知己》,《东西爱情故事》
重庆瑜影画文化传播有限责任公司、安徽艺客影视文化传媒有限公司：《铁甲少年》,《我是美术生之星空筑梦》,《星之守望》
南派传奇影视有限公司：《天才厨神》,《寻找失散的恋人》
世纪领略影业（北京）有限公司、星灿宇宸影业（青岛）有限公司：《无可替代》,《求求你，站出来》
甘肃田野之光影视文化发展有限公司：《再见故乡桥》,《我不能说》
风云（深圳）传媒有限公司：《岌岌之爱》,《海棠莉舍的婚房》
四川合力行文化传媒有限公司：《敬启者》,《狂萌飙车》
南充梅尔西文化传媒有限责任公司、北京梅尔西文化传媒有限公司：《因你而动》,《村民评测》
陕西嘉轩天赋影视文化有限公司：《南沧·龙珠传》,《托起明天的太阳》
山东橙果影视文化发展有限公司：《寻狗奇兵》,《绝世抠神》
广州新片场影视传媒有限公司：《乌龙院之七鲜鱼丸》,《河兽》
北京星光星成文化传媒有限公司：《我的高中三年》,《记忆中的那片海》
长影集团有限责任公司：《姐姐》,《啤酒》
双龙飞木（佛山）影业有限公司：《眼泪日记》,《“一个人”的球队》
牛我影业（深圳）有限公司：《积木游戏》,《安妮的旅行》


In [ ]:
# Load Model
model weights
content
predictions